# Power Outages Data Analysis

**Name**: Aditya Agrawal

**Website Link**: https://adia32.github.io/poweroutages/

### **Question**: When do major power outages tend to occur?

## Code

In [63]:
import pandas as pd
import numpy as np
import os
import folium # needed to visualize geospatial data
import matplotlib.pyplot as plt # needed to plot data 


import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Cleaning and EDA

### Cleaning

Importing the data and removing certain columns and rows. Data was given as excel sheet so using relevant commands to import data and format as much as possible with the built in read_excel function. 

In [64]:
# The real header of the data was at row 6 and row 7 was units not actual data so:
outages = pd.read_excel("outage.xlsx", header=5, usecols="C:BE", skiprows=[6])

In [65]:
outages.shape

(1534, 55)

In [66]:
outages.info()  # doing a quick scan of data, and checking Dtype and Null Counts

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 0 to 1533
Data columns (total 55 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   YEAR                     1534 non-null   int64         
 1   MONTH                    1525 non-null   float64       
 2   U.S._STATE               1534 non-null   object        
 3   POSTAL.CODE              1534 non-null   object        
 4   NERC.REGION              1534 non-null   object        
 5   CLIMATE.REGION           1528 non-null   object        
 6   ANOMALY.LEVEL            1525 non-null   float64       
 7   CLIMATE.CATEGORY         1525 non-null   object        
 8   OUTAGE.START.DATE        1525 non-null   datetime64[ns]
 9   OUTAGE.START.TIME        1525 non-null   object        
 10  OUTAGE.RESTORATION.DATE  1476 non-null   datetime64[ns]
 11  OUTAGE.RESTORATION.TIME  1476 non-null   object        
 12  CAUSE.CATEGORY           1534 non-

the dataframe stores "OUTAGE.START.DATE" as timestamp objects already and "OUTAGE.START.TIME" as datetime.time objects, to make them compatible with each other we covert them to datetime and timedelta objects respectively. then we add them simply and assign to new column. tbh converting OUTAGE.START.DATE into datetime is optional because its already in a workable format. this is just for extra safety.

In [67]:
# Merging columns by putting them both into datetime objects and adding.
outages["OUTAGE.START"] = pd.to_datetime(outages["OUTAGE.START.DATE"]) + pd.to_timedelta(
    outages["OUTAGE.START.TIME"].astype(str)
)

# Same as above
outages["OUTAGE.RESTORATION"] = pd.to_datetime(
    outages["OUTAGE.RESTORATION.DATE"]
) + pd.to_timedelta(outages["OUTAGE.RESTORATION.TIME"].astype(str))

# Dropping merged
outages.drop(
    columns=[
        "OUTAGE.START.DATE",
        "OUTAGE.START.TIME",
        "OUTAGE.RESTORATION.DATE",
        "OUTAGE.RESTORATION.TIME",
    ],
    inplace=True,
)
outages[["OUTAGE.START", "OUTAGE.RESTORATION"]]  # Display new columns

,OUTAGE.START,OUTAGE.RESTORATION
0,2011-07-01 17:00:00,2011-07-03 20:00:00
1,2014-05-11 18:38:00,2014-05-11 18:39:00
2,2010-10-26 20:00:00,2010-10-28 22:00:00
3,2012-06-19 04:30:00,2012-06-20 23:00:00
4,2015-07-18 02:00:00,2015-07-19 07:00:00
...,...,...
1529,2011-12-06 08:00:00,2011-12-06 20:00:00
1530,NaT,NaT
1531,2009-08-29 22:54:00,2009-08-29 23:53:00
1532,2009-08-29 11:00:00,2009-08-29 14:01:00


In [68]:
# create new columns for day of the week, and hour. used in univariate analysis later.
outages["DAY.OF.WEEK"] = outages["OUTAGE.START"].dt.dayofweek
outages["HOUR"] = outages["OUTAGE.START"].dt.hour
outages = outages[
    outages.columns[:2].to_list()
    + ["DAY.OF.WEEK", "HOUR"]
    + outages.columns[2:-2].to_list()
]

In [69]:
print(outages.head().to_markdown(index=False))  # for website.

|   YEAR |   MONTH |   DAY.OF.WEEK |   HOUR | U.S._STATE   | POSTAL.CODE   | NERC.REGION   | CLIMATE.REGION     |   ANOMALY.LEVEL | CLIMATE.CATEGORY   | CAUSE.CATEGORY     | CAUSE.CATEGORY.DETAIL   |   HURRICANE.NAMES |   OUTAGE.DURATION |   DEMAND.LOSS.MW |   CUSTOMERS.AFFECTED |   RES.PRICE |   COM.PRICE |   IND.PRICE |   TOTAL.PRICE |   RES.SALES |   COM.SALES |   IND.SALES |   TOTAL.SALES |   RES.PERCEN |   COM.PERCEN |   IND.PERCEN |   RES.CUSTOMERS |   COM.CUSTOMERS |   IND.CUSTOMERS |   TOTAL.CUSTOMERS |   RES.CUST.PCT |   COM.CUST.PCT |   IND.CUST.PCT |   PC.REALGSP.STATE |   PC.REALGSP.USA |   PC.REALGSP.REL |   PC.REALGSP.CHANGE |   UTIL.REALGSP |   TOTAL.REALGSP |   UTIL.CONTRI |   PI.UTIL.OFUSA |   POPULATION |   POPPCT_URBAN |   POPPCT_UC |   POPDEN_URBAN |   POPDEN_UC |   POPDEN_RURAL |   AREAPCT_URBAN |   AREAPCT_UC |   PCT_LAND |   PCT_WATER_TOT |   PCT_WATER_INLAND | OUTAGE.START        | OUTAGE.RESTORATION   |
|-------:|--------:|--------------:|-------:|:------------

### Univariate Analysis

In [70]:
# Below I just plotted a few columns to see the data better for my personal insight.
# I put two of them on the website and explain them more

In [71]:
# Create a bar chart of the cause category details
fig_cause = px.bar(
    data_frame=outages,
    x="CAUSE.CATEGORY.DETAIL",
    title="Distribution of Power Outage Causes",
)
fig_cause.update_xaxes(title="Cause Category Detail")
fig_cause.update_yaxes(title="Count")
fig_cause.write_html("assets/cause_chart.html", include_plotlyjs="cdn")

# Display the plot
fig_cause.show()

In [72]:
# Calculate the count of power outages for each day of the week
day_of_week_counts = outages["DAY.OF.WEEK"].value_counts()

# Create a bar chart of the day of the week
fig_day_of_week = px.bar(
    x=day_of_week_counts.index,
    y=day_of_week_counts.values,
    title="Distribution of Power Outages by Day of Week",
)
fig_day_of_week.update_xaxes(title="Day of Week")
fig_day_of_week.update_yaxes(title="Count")

fig_day_of_week.show()

In [73]:
# Power Outage Duration Distribution (Histogram)
fig_hist = px.histogram(
    outages, x="OUTAGE.DURATION", nbins=30, title="Distribution of Power Outage Duration"
)
fig_hist.update_layout(xaxis_title="Outage Duration (minutes)", yaxis_title="Frequency")
fig_hist.show()
fig_hist.write_html("assets/outage_chart.html", include_plotlyjs="cdn")

In [74]:
# Calculate the count of power outages for each month
month_counts = outages['MONTH'].value_counts().sort_index()

# Create a bar chart of the month
fig_month = px.bar(x=month_counts.index, y=month_counts.values, title='Distribution of Power Outages by Month')
fig_month.update_xaxes(title='Month')
fig_month.update_yaxes(title='Count')

fig_month.show()
fig_hist.write_html("assets/month_only_chart.html", include_plotlyjs="cdn")

### Bivariate Analysis

In [75]:
# Create a box plot of the day of the week
fig_day_of_week = px.box(
    data_frame=outages,
    x="DAY.OF.WEEK",
    title="Distribution of Power Outages by Day of Week",
    y="OUTAGE.DURATION",
)
fig_day_of_week.update_xaxes(title="Day of Week")
fig_day_of_week.update_yaxes(title="Outage Duration (minutes)")

fig_day_of_week.show()

In [76]:
# Create a box plot of the month
fig_month = px.box(
    data_frame=outages,
    x="MONTH",
    title="Distribution of Power Outages by Month",
    y="OUTAGE.DURATION",
)
fig_month.update_xaxes(title="Month")
fig_month.update_yaxes(title="Outage Duration (minutes)")
fig_month.write_html("assets/month_boxplot.html", include_plotlyjs="cdn")

fig_month.show()


In [77]:
# Create a scatter plot
fig = px.scatter(
    outages,
    x="MONTH",
    y="OUTAGE.DURATION",
    title="Outage Duration by Month",
    labels={
        "MONTH": "Month of the Year",
        "OUTAGE.DURATION": "Duration of Outage (Minutes)",
    },
    category_orders={"MONTH": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]},
)
fig.show()
# okay box clearly works better

In [78]:
# Bivariate analysis of outage duration vs. month
fig = px.scatter(outages, x="YEAR", y="DEMAND.LOSS.MW")
fig.show()

In [79]:
# Bivariate analysis of outage duration vs. month
fig = px.scatter(outages, x="OUTAGE.DURATION", y="DEMAND.LOSS.MW")
fig.show()

### Interesting Aggregates

In [80]:
# Aggregate analysis: average and number of outage duration by month
outage_stats_by_month = (
    outages.groupby("MONTH")["OUTAGE.DURATION"].agg(["mean", "count"]).reset_index()
)
display(outage_stats_by_month)
print(outage_stats_by_month.to_markdown(index=False))
# Explanation: This table shows the average outage duration and count for each month. It can provide insights into whether outages tend to be longer or shorter in certain months.

,MONTH,mean,count
0,1.0,3387.947368,133
1,2.0,2497.143939,132
2,3.0,3265.893617,94
3,4.0,1493.859813,107
4,5.0,2077.294118,119
5,6.0,1948.403226,186
6,7.0,2218.988701,177
7,8.0,2428.476821,151
8,9.0,4294.521739,92
9,10.0,3600.935185,108


|   MONTH |    mean |   count |
|--------:|--------:|--------:|
|       1 | 3387.95 |     133 |
|       2 | 2497.14 |     132 |
|       3 | 3265.89 |      94 |
|       4 | 1493.86 |     107 |
|       5 | 2077.29 |     119 |
|       6 | 1948.4  |     186 |
|       7 | 2218.99 |     177 |
|       8 | 2428.48 |     151 |
|       9 | 4294.52 |      92 |
|      10 | 3600.94 |     108 |
|      11 | 1728.16 |      69 |
|      12 | 3293.79 |     108 |


### Assessment of Missingness

#testing something
<details>
<summary>Click to toggle contents of `code`</summary>

```
CODE!
```
</details>

In [81]:
#checking to see what column in the dataset has non-trivial missingness
outages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 0 to 1533
Data columns (total 55 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   YEAR                   1534 non-null   int64         
 1   MONTH                  1525 non-null   float64       
 2   DAY.OF.WEEK            1525 non-null   float64       
 3   HOUR                   1525 non-null   float64       
 4   U.S._STATE             1534 non-null   object        
 5   POSTAL.CODE            1534 non-null   object        
 6   NERC.REGION            1534 non-null   object        
 7   CLIMATE.REGION         1528 non-null   object        
 8   ANOMALY.LEVEL          1525 non-null   float64       
 9   CLIMATE.CATEGORY       1525 non-null   object        
 10  CAUSE.CATEGORY         1534 non-null   object        
 11  CAUSE.CATEGORY.DETAIL  1063 non-null   object        
 12  HURRICANE.NAMES        72 non-null     object        
 13  OUT

In [82]:
# Subset dataframe and create a binary column indicating whether `DEMAND.LOSS.MW` is missing
# We are going to analyze with 'OUTAGE.DURATION' and 'YEAR'.
permutation_df = outages.copy()
permutation_df = permutation_df[["DEMAND.LOSS.MW", "OUTAGE.DURATION", "YEAR"]]

permutation_df["DEMAND.LOSS.MW_isnull"] = permutation_df["DEMAND.LOSS.MW"].isnull()
permutation_df

,DEMAND.LOSS.MW,OUTAGE.DURATION,YEAR,DEMAND.LOSS.MW_isnull
0,NaN,3060.0,2011,True
1,NaN,1.0,2014,True
2,NaN,3000.0,2010,True
3,NaN,2550.0,2012,True
4,250.0,1740.0,2015,False
...,...,...,...,...
1529,155.0,720.0,2011,False
1530,1650.0,NaN,2006,False
1531,84.0,59.0,2009,False
1532,373.0,181.0,2009,False


In [83]:
# Check the missingness dependency with `OUTAGE.DURATION`
grouped = permutation_df.groupby("DEMAND.LOSS.MW_isnull")["OUTAGE.DURATION"].mean()

obs_1 = grouped.diff().iloc[-1]
test_stats_1 = []
shuffled = permutation_df.copy()
for _ in range(1000):
    shuffled["OUTAGE.DURATION"] = np.random.permutation(shuffled["OUTAGE.DURATION"])
    shuffled_group = shuffled.groupby("DEMAND.LOSS.MW_isnull")["OUTAGE.DURATION"].mean()
    test_stat = shuffled_group.diff().iloc[-1]
    test_stats_1.append(test_stat)

pval_1 = (np.array(test_stats_1) > obs_1).mean()

# Print the results
print(f"P-value for OUTAGE.DURATION: {pval_1}")

P-value for OUTAGE.DURATION: 0.332


In [84]:
fig = px.histogram(
    test_stats_1,
    nbins=30,
    labels={"value": "Test Statistic"},
    title="Permutation Test: OUTAGE.DURATION",
    histnorm='probability'
)
fig.add_vline(
    x=obs_1,
    line_dash="dash",
    line_color="red",
    annotation_text=f'<span style="color:red">Observed Statistic = {round(obs_1, 2)}</span>',
    annotation_position="top right",
)
fig.show()
fig.write_html("assets/missing1.html", include_plotlyjs="cdn")

In [85]:
# Check the missingness dependency with `YEAR`
grouped = permutation_df.groupby("DEMAND.LOSS.MW_isnull")["YEAR"].mean()

obs_2 = grouped.diff().iloc[-1]
test_stats_2 = []
shuffled = permutation_df.copy()
for _ in range(1000):
    shuffled["YEAR"] = np.random.permutation(shuffled["YEAR"])
    shuffled_group = shuffled.groupby("DEMAND.LOSS.MW_isnull")["YEAR"].mean()
    test_stat = shuffled_group.diff().iloc[-1]
    test_stats_2.append(test_stat)

pval_2 = (np.array(test_stats_2) > obs_2).mean()

# Print the results
print(f"P-value for YEAR: {pval_2}")

P-value for YEAR: 0.0


In [86]:
fig = px.histogram(
    test_stats_2,
    nbins=30,
    labels={"value": "Test Statistic"},
    title="Permutation Test: YEAR",
    histnorm='probability'
)
fig.add_vline(
    x=obs_2,
    line_dash="dash",
    line_color="red",
    annotation_text=f'<span style="color:red">Observed Statistic = {round(obs_2, 2)}</span>',
    annotation_position="top right",
)
fig.show()
fig.write_html("assets/missing2.html", include_plotlyjs="cdn")

### Hypothesis Testing

Null Hypothesis (H0): Major power outages are equally likely to occur in winter months (December, January, February), summer months (June, July, August), and other months.

Alternative Hypothesis (H1): Major power outages are more likely to occur during winter and summer months compared to other months.

Test Statistic: The difference between the number of outages during winter and summer months and the number of outages during other months.

Significance Level: We will use a significance level of 0.05. If the p-value is less than 0.05, we reject the null hypothesis.




In [87]:
# Create a binary column indicating whether the outage is in winter or summer
df_winter_summer = outages[outages['MONTH'].isin([12, 1, 2, 6, 7, 8])]
df_other = outages[~outages['MONTH'].isin([12, 1, 2, 6, 7, 8])]
observed_diff = len(df_winter_summer) - len(df_other)

# Permutation test
n_repetitions = 5000

diffs = []
for _ in range(n_repetitions):
    
    # Shuffle the binary column indicating winter or summer
    shuffled_col = np.random.choice([True, False], size=len(outages))
    
    # Compute the differences
    shuffled = outages.assign(is_winter_summer=shuffled_col)
    shuffled_winter_summer = shuffled[shuffled['is_winter_summer'] == True]
    shuffled_other = shuffled[shuffled['is_winter_summer'] == False]
    diff = len(shuffled_winter_summer) - len(shuffled_other)
    
    diffs.append(diff)

p_val = (np.array(diffs) > observed_diff).mean()

print(f"P-value: {p_val}")


P-value: 0.0


In [88]:
# Convert diffs to DataFrame for compatibility with plotly
diffs_df = pd.DataFrame(diffs, columns=["Difference"])

fig = px.histogram(diffs_df, x="Difference")
fig.add_vline(
    x=observed_diff,
    line_color="red",
    line_dash="dash",
    annotation_text="Observed Difference",
)
fig.show()
fig.write_html('assets/hyptesting.html', include_plotlyjs='cdn')